**In this little tutorial we'll get deeper into SVD based on user-item concept**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
ratings = pd.read_csv('/content/drive/My Drive/ratings_small.csv') # all this data sets you'll find in my repo 
id_map = pd.read_csv('/content/id_title.csv')

In [26]:
id_map.head()
ratings.head() # let's remember how they look

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [3]:
id_map2 = id_map.set_index('movieId')[['title']]

In [13]:
id_map2.head()

,title
movieId,
1,Toy Story
2,Jumanji
3,Grumpier Old Men
4,Waiting to Exhale
5,Father of the Bride Part II


In [8]:
ratings_2 = ratings[ratings.movieId.isin(id_map2.index)]

In [29]:
user_item = ratings_2.groupby(['userId', 'movieId'])['rating'].first().unstack(fill_value = 0.)
user_item.head() # zero values mean that a particular user didn't rate a particular movie

movieId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,35,36,37,38,39,40,41,...,155611,155820,156025,156387,156607,156609,156726,157200,157296,157407,157667,158238,158314,158528,158956,159093,159462,159690,159755,159858,159972,160080,160271,160438,160440,160563,160565,160567,160590,160656,160718,161084,161155,161594,161830,161918,161944,162542,162672,163949
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
user_item.loc[42].sort_values(ascending=False).head() # take an example

movieId
589     5.0
296     5.0
1200    5.0
1196    5.0
480     5.0
Name: 42, dtype: float64

In [32]:
id_map2.loc[user_item.loc[42].sort_values(ascending=False).head().index] # a list of movies which User_42 has liked most

,title
movieId,
589,Terminator 2: Judgment Day
296,Pulp Fiction
1200,Aliens
1196,The Empire Strikes Back
480,Jurassic Park


In [16]:
from numpy.linalg import svd 
U, S, Vt = svd(user_item, full_matrices=False)
rank = [50,100] # you can experiment with the rank 
for r in rank:
  matrix = U[:, :r] @ np.diag(S)[:r,:r] @ Vt[:r,:]
  if r == 50:
    pred_df_50 = pd.DataFrame(matrix, columns=user_item.columns, index=user_item.index)
  else:
    pred_df_100 = pd.DataFrame(matrix, columns=user_item.columns, index=user_item.index)

In [37]:
pred_df_100.loc[42].sort_values(ascending=False).head() # so our predicted df thinks, that this movies would be high rated by User_42

movieId
260     5.234493
1196    4.653261
2571    4.637456
1198    4.312162
1210    4.216438
Name: 42, dtype: float64

In [41]:
user_item.loc[42,pred_df_100.loc[42].sort_values(ascending=False).head().index] 
# let's see true ratings, only movie 2571 was considered wrong

movieId
260     4.5
1196    5.0
318     5.0
2571    3.5
1198    5.0
Name: 42, dtype: float64

In [38]:
id_map2.loc[pred_df_100.loc[42].sort_values(ascending=False).head().index] # titles for this movies

,title
movieId,
260,Star Wars
1196,The Empire Strikes Back
318,The Shawshank Redemption
2571,The Matrix
1198,Raiders of the Lost Ark


In [138]:
def get_recommendations(userId):
  """
  Recommendation of movies for User, which he/she hasn't rated yet
  based on SVD of ratings data set
  """
  movies_rated_by_user = user_item.loc[userId]
  movies_zero_rated = movies_rated_by_user[movies_rated_by_user == 0.].index
  movies_titles = id_map2.loc[movies_zero_rated]

  movies_recommended_for_user = pred_df_100.loc[userId]
  movies_zero_recommended = movies_recommended_for_user.loc[movies_zero_rated]
  df = pd.DataFrame({'predicted_ratings':movies_zero_recommended.values, 'recommended_movie':movies_titles.title})

  movies_highly_rated_by_user = movies_rated_by_user.sort_values(ascending=False).index
  movies_highly_rated_titles = id_map2.loc[movies_highly_rated_by_user].reset_index().title.head(10)

  df = df.sort_values(by='predicted_ratings',ascending=False).reset_index().head(10)
  df['highly_rated_movies'] = movies_highly_rated_titles
  return df

In [140]:
get_recommendations(42)

,movieId,predicted_ratings,recommended_movie,highly_rated_movies
0,593,2.445404,The Silence of the Lambs,Terminator 2: Judgment Day
1,1270,2.189468,Back to the Future,Pulp Fiction
2,1,2.084618,Toy Story,Aliens
3,6539,1.857068,Pirates of the Caribbean: The Curse of the Bla...,The Empire Strikes Back
4,54286,1.853336,The Bourne Ultimatum,Jurassic Park
5,1240,1.762529,The Terminator,The Lord of the Rings: The Fellowship of the Ring
6,3578,1.754469,Gladiator,Raiders of the Lost Ark
7,1197,1.550838,The Princess Bride,The Fugitive
8,377,1.513070,Speed,E.T. the Extra-Terrestrial
9,150,1.505294,Apollo 13,The Shawshank Redemption


In [137]:
get_recommendations(3) # I think it's a pretty good recommendation despite the fact that predicted ratings are low

,movieId,predicted_ratings,recommended_movie,highly_rated_movies
0,2571,1.754071,The Matrix,Fight Club
1,1704,1.740603,Good Will Hunting,Forrest Gump
2,1,1.494359,Toy Story,The Shawshank Redemption
3,4993,1.484978,The Lord of the Rings: The Fellowship of the Ring,The Princess Bride
4,5952,1.397031,The Lord of the Rings: The Two Towers,Requiem for a Dream
5,480,1.346502,Jurassic Park,Flags of Our Fathers
6,260,1.342534,Star Wars,Titanic 1997
7,2329,1.295022,American History X,Letters from Iwo Jima
8,590,1.087855,Dances with Wolves,Pulp Fiction
9,150,1.071987,Apollo 13,The White Stripes: Under Great White Northern ...


In [141]:
# this function doesn't take into account movies, which User has already rated
def get_recommendations_2(userId): 
  """
  Recommendation of movies for User based on SVD of ratings data set
  """
  movies_rated_by_user = user_item.loc[userId]
  movies_highly_rated_by_user = movies_rated_by_user[movies_rated_by_user > 3].index

  movies_recommended_for_user = pred_df_100.loc[userId]
  movies_highly_recommend_for_user = movies_recommended_for_user[movies_recommended_for_user > 3].index
  
  # leave only those, which User haven't rated yet
  result = list(set(movies_highly_recommend_for_user) - set(movies_highly_rated_by_user))
  if result != []:
    return result, list(id_map2.loc[result].title.values)
  else:
    return 0, 0

In [142]:
for idx in range(1,100):
  indices, movies = get_recommendations_2(idx)
  if (indices, movies) != (0, 0):
    print('UserId: {}, movies: {}'.format(idx, movies)) 
# as you can see our model isn't great: it can't recommend smth new for some users

UserId: 2, movies: ['The Silence of the Lambs', 'The Fugitive', 'Forrest Gump', 'Aladdin']
UserId: 3, movies: ["Schindler's List"]
UserId: 4, movies: ['Batman 1989', 'Beauty and the Beast 1991', 'True Lies', 'Dances with Wolves']
UserId: 7, movies: ['Forrest Gump', 'Terminator 2: Judgment Day', 'The Princess Bride']
UserId: 15, movies: ['42 Up', 'GoldenEye', 'The World Is Not Enough', 'Avengers: Age of Ultron', 'Zootopia', 'Strange Brew', 'Ant-Man', 'Go', 'Leaving Las Vegas', 'Innerspace', 'Blue Velvet', 'TRON: Legacy', 'So I Married an Axe Murderer', 'Planet of the Apes 2001', 'Election', 'Captain America: Civil War', 'eXistenZ', 'Willy Wonka & the Chocolate Factory', 'Welcome to the Dollhouse', 'Harry Potter and the Deathly Hallows: Part 2', 'The Mexican', 'Uncle Buck', 'The Peacemaker', 'Frankenstein', 'Magnolia', 'Rush Hour 2', 'Beautiful Girls', 'Collateral', 'Heavy Metal', 'The Gods Must Be Crazy', "The Devil's Advocate", 'The NeverEnding Story', 'Gattaca', 'Blade', 'The Wrong Tr

In [145]:
indices = user_item.loc[3].sort_values(ascending=False).head(10).index # real data
_, movies = get_recommendations_2(3) # recommended 
list(id_map2.loc[indices].title.values), 'Recommendations:', list(movies)

(['Fight Club',
  'Forrest Gump',
  'The Shawshank Redemption',
  'The Princess Bride',
  'Requiem for a Dream',
  'Flags of Our Fathers',
  'Titanic 1997',
  'Letters from Iwo Jima',
  'Pulp Fiction',
  'The White Stripes: Under Great White Northern Lights'],
 'Recommendations:',
 ["Schindler's List"])

In comparison with the previous model this model only recommends one movie "Schindler's List" for User 3. That's the main drawback of this model.